In [10]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms

In [15]:
class SquareDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.X = torch.randint(255, (size, 9), dtype=torch.float)

        real_w = torch.tensor([[1,1,1,0,0,0,0,0,0],
                               [0,0,0,1,1,1,0,0,0],
                               [0,0,0,0,0,0,1,1,1]], 
                               dtype=torch.float)

        self.Y = torch.argmax(self.X.mm(real_w.t()), 1)

    def __getitem__(self, index):
        y = torch.zeros(3, dtype=torch.float) \
                 .scatter_(0, self.Y[index], 1)
        return (self.X[index], y)

    def __len__(self):
        return self.size

In [20]:
squares = SquareDataset(256)
print(squares[34])
print(squares[254])

(tensor([125.,  92., 214.,  37.,  86.,   4., 185.,  45.,  17.]), tensor([1., 0., 0.]))
(tensor([153.,  62.,  91., 238., 105., 107.,  46.,  41., 210.]), tensor([0., 1., 0.]))


In [25]:
dataloader = DataLoader(squares, batch_size=16)

In [28]:
for batch, (X, Y) in enumerate(dataloader):
    print(X)
    print(Y)
    break

tensor([[246., 173.,  51., 140.,  39.,  72., 101., 147., 204.],
        [145., 182., 111., 110., 206., 213., 221.,  37., 215.],
        [192.,   1.,  40., 193.,  71., 237., 229.,   5., 230.],
        [132., 233.,  62., 111.,   7., 222., 117., 205.,  74.],
        [ 31., 143., 251., 148.,  25., 254., 114., 131., 139.],
        [  7., 198.,  21., 148.,  25., 138., 174., 186., 236.],
        [252.,  53.,  10., 189., 191., 128., 159.,  75., 149.],
        [197., 193., 190., 137.,  20.,  86.,  75.,  67., 164.],
        [224., 136.,   1., 214., 189., 170.,  36., 114.,  51.],
        [162.,  60.,   1.,  45.,  31.,  18., 202.,  42.,  52.],
        [ 96.,  18., 105., 229., 114., 235., 200., 111.,  88.],
        [195., 179.,  87.,  77.,  11., 100., 224., 158., 214.],
        [ 41., 105.,  18., 166., 235., 112., 169.,  56.,  45.],
        [207., 159., 123.,  14.,  96., 100., 173., 116.,  81.],
        [ 97., 197., 232., 194.,  93., 118.,  33.,  67.,  64.],
        [207., 241., 137., 105.,  17., 1

In [30]:
digits = datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Lambda(lambda x: x.reshape(28*28))
                    ]),
                    target_transform=transforms.Compose([
                        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, y, 1))
                    ])
                 )
dataloader = DataLoader(digits, batch_size=10, shuffle=True)

In [31]:
for batch, (X, Y) in enumerate(dataloader):
    print(X)
    print(Y)
    break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
